In [12]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [13]:
import warnings

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import os

In [14]:
warnings.filterwarnings('ignore')

In [19]:
# Initialize Spark session
spark = SparkSession.builder.appName("CrimePatrol").getOrCreate()

# Load dataset
crime_data = spark.read.csv("Major_Crime_Indicators_Open_Data.csv", header=True, inferSchema=True)

# Explore the dataset
crime_data.printSchema()


root
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- OBJECTID: integer (nullable = true)
 |-- EVENT_UNIQUE_ID: string (nullable = true)
 |-- REPORT_DATE: string (nullable = true)
 |-- OCC_DATE: string (nullable = true)
 |-- REPORT_YEAR: integer (nullable = true)
 |-- REPORT_MONTH: string (nullable = true)
 |-- REPORT_DAY: integer (nullable = true)
 |-- REPORT_DOY: integer (nullable = true)
 |-- REPORT_DOW: string (nullable = true)
 |-- REPORT_HOUR: integer (nullable = true)
 |-- OCC_YEAR: integer (nullable = true)
 |-- OCC_MONTH: string (nullable = true)
 |-- OCC_DAY: integer (nullable = true)
 |-- OCC_DOY: integer (nullable = true)
 |-- OCC_DOW: string (nullable = true)
 |-- OCC_HOUR: integer (nullable = true)
 |-- DIVISION: string (nullable = true)
 |-- LOCATION_TYPE: string (nullable = true)
 |-- PREMISES_TYPE: string (nullable = true)
 |-- UCR_CODE: integer (nullable = true)
 |-- UCR_EXT: integer (nullable = true)
 |-- OFFENCE: string (nullable = true)
 |-- M

In [20]:
crime_data = crime_data.drop(crime_data.X)
crime_data = crime_data.drop(crime_data.Y)
crime_data = crime_data.drop(crime_data.OBJECTID)
crime_data = crime_data.drop(crime_data.EVENT_UNIQUE_ID)
crime_data = crime_data.drop(crime_data.REPORT_DATE)
crime_data = crime_data.drop(crime_data.REPORT_YEAR)
crime_data = crime_data.drop(crime_data.REPORT_MONTH)
crime_data = crime_data.drop(crime_data.REPORT_DAY)
crime_data = crime_data.drop(crime_data.REPORT_DOY)
crime_data = crime_data.drop(crime_data.REPORT_DOW)
crime_data = crime_data.drop(crime_data.REPORT_HOUR)
crime_data = crime_data.drop(crime_data.OCC_YEAR)
crime_data = crime_data.drop(crime_data.OCC_MONTH)
crime_data = crime_data.drop(crime_data.OCC_DAY)
crime_data = crime_data.drop(crime_data.OCC_DATE)
crime_data = crime_data.drop(crime_data.OCC_DOY)
crime_data = crime_data.drop(crime_data.DIVISION)
crime_data = crime_data.drop(crime_data.LOCATION_TYPE)
crime_data = crime_data.drop(crime_data.PREMISES_TYPE)
crime_data = crime_data.drop(crime_data.UCR_CODE)
crime_data = crime_data.drop(crime_data.UCR_EXT)
crime_data = crime_data.drop(crime_data.OFFENCE)
crime_data = crime_data.drop(crime_data.MCI_CATEGORY)
crime_data = crime_data.drop(crime_data.HOOD_158)
crime_data = crime_data.drop(crime_data.NEIGHBOURHOOD_158)
crime_data = crime_data.drop(crime_data.HOOD_140)
crime_data = crime_data.drop(crime_data.NEIGHBOURHOOD_140)

crime_data.printSchema()

root
 |-- OCC_DOW: string (nullable = true)
 |-- OCC_HOUR: integer (nullable = true)
 |-- LONG_WGS84: double (nullable = true)
 |-- LAT_WGS84: double (nullable = true)



In [23]:
#drop invalid entries

#drop entries with latitude or longitued of 0 whicha re invalid in our data
crime_data = crime_data.filter(crime_data["LAT_WGS84"] != 0)
crime_data = crime_data.filter(crime_data["LONG_WGS84"] != 0)

#drop entries that were inputted with an hour outside the 24 hr clock
crime_data = crime_data.filter((col("OCC_HOUR") >= 0) & (col("OCC_HOUR") <= 23))

#drop entries without a valid day of the week
valid_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
crime_data = crime_data.filter(trim(crime_data["OCC_DOW"]).isin(valid_days))


crime_data.show(15)

crime_data.write.option("header", "true").csv()

+----------+--------+-----------------+----------------+
|   OCC_DOW|OCC_HOUR|       LONG_WGS84|       LAT_WGS84|
+----------+--------+-----------------+----------------+
|Wednesday |       0|-79.3842059245105|43.6707982509655|
|Wednesday |       0|-79.3457948187012|43.7036843035894|
|Wednesday |      10|-79.3791313378142|43.6459808612379|
|Wednesday |       0|-79.3832001629001|43.6543126531725|
|Wednesday |       8|-79.5137965538313|43.7198240863369|
|Wednesday |       8| -79.421673390898|43.7923239909928|
|Wednesday |       9|-79.3383011325343|43.6741310288285|
|Wednesday |       4|-79.4839287578705|43.7343540636839|
|Wednesday |       4|-79.4019981704324|43.6476071214313|
|Wednesday |      16|-79.4145751194734|43.7716977360472|
|Wednesday |       3|-79.1740333794421|43.7613933196769|
|Tuesday   |      21|-79.4765867987994|43.7643128234304|
|Wednesday |       3|-79.4062302421994|43.7276563231215|
|Wednesday |      18|-79.3255139812618|43.6908142095598|
|Thursday  |       0|-79.286726

23/12/10 12:42:46 ERROR FileOutputCommitter: Mkdirs failed to create file:/cleaned_data.csv/_temporary/0
23/12/10 12:42:47 ERROR Executor: Exception in task 0.0 in stage 11.0 (TID 32)
java.io.IOException: Mkdirs failed to create file:/cleaned_data.csv/_temporary/0/_temporary/attempt_202312101242471045332080179448147_0011_m_000000_32 (exists=false, cwd=file:/Users/samtomecek/Documents/GitHub/PatrolOptimizer-BigDataAnalysis)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStreamWriter(CodecStreams.scala:92)
	at or

Py4JJavaError: An error occurred while calling o486.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 11.0 failed 1 times, most recent failure: Lost task 3.0 in stage 11.0 (TID 35) (192.168.1.82 executor driver): java.io.IOException: Mkdirs failed to create file:/cleaned_data.csv/_temporary/0/_temporary/attempt_202312101242471045332080179448147_0011_m_000003_35 (exists=false, cwd=file:/Users/samtomecek/Documents/GitHub/PatrolOptimizer-BigDataAnalysis)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.spark.sql.execution.datasources